# Regression Week 1: Simple Linear Regression

In this notebook we will use data on house sales in King County to predict house prices using simple (one input) linear regression. You will:
* Write a function to compute the Simple Linear Regression weights using the closed form solution
* Write a function to make predictions of the output given the input feature
* Turn the regression around to predict the input given the output
* Compare two different models for predicting house prices

In this notebook you will be provided with some already complete code as well as some code that you should complete yourself in order to answer quiz questions. The code we provide to complete is optional and is there to assist you with solving the problems but feel free to ignore the helper code and write your own.

In [58]:
# import graphlab
import pandas as pd
import numpy as np

# Load house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [59]:
dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float, 'grade':int, 'yr_renovated':int, 
              'price':float, 'bedrooms':float, 'zipcode':str, 'long':float, 'sqft_lot15':float, 'sqft_living':float, 
              'floors':str, 'condition':int, 'lat':float, 'date':str, 'sqft_basement':int, 'yr_built':int, 'id':str, 
              'sqft_lot':int, 'view':int}

In [60]:
type(dtype_dict)

dict

In [61]:
# sales = graphlab.SFrame('kc_house_data.gl/')
df_house_data = pd.read_csv("../../../../data/coursera_regression/kc_house_data.csv", dtype= dtype_dict)
df_house_train = pd.read_csv("../../../../data/coursera_regression/kc_house_train_data.csv", dtype= dtype_dict)
df_house_test = pd.read_csv("../../../../data/coursera_regression/kc_house_test_data.csv", dtype= dtype_dict)

In [62]:
df_house_data.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,0114101516,20140528T000000,310000.0,3.0,1.0,1430.0,19901,1.5,0,0,...,7,1430,0,1927,0,98028,47.7558,-122.229,1780.0,12697.0
1,9297300055,20150124T000000,650000.0,4.0,3.0,2950.0,5000,2,0,3,...,9,1980,970,1979,0,98126,47.5714,-122.375,2140.0,4000.0
2,1202000200,20141103T000000,233000.0,3.0,2.0,1710.0,4697,1.5,0,0,...,6,1710,0,1941,0,98002,47.3048,-122.218,1030.0,4705.0
3,8562750320,20141110T000000,580500.0,3.0,2.5,2320.0,3980,2,0,0,...,8,2320,0,2003,0,98027,47.5391,-122.070,2580.0,3980.0
4,7589200193,20141110T000000,535000.0,3.0,1.0,1090.0,3000,1.5,0,0,...,8,1090,0,1929,0,98117,47.6889,-122.375,1570.0,5080.0


In [63]:
df_house_train.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3.0,1.00,1180.0,5650,1,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340.0,5650.0
1,6414100192,20141209T000000,538000.0,3.0,2.25,2570.0,7242,2,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690.0,7639.0
2,5631500400,20150225T000000,180000.0,2.0,1.00,770.0,10000,1,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720.0,8062.0
3,2487200875,20141209T000000,604000.0,4.0,3.00,1960.0,5000,1,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360.0,5000.0
4,1954400510,20150218T000000,510000.0,3.0,2.00,1680.0,8080,1,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800.0,7503.0


In [64]:
df_house_test.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,0114101516,20140528T000000,310000.0,3.0,1.0,1430.0,19901,1.5,0,0,...,7,1430,0,1927,0,98028,47.7558,-122.229,1780.0,12697.0
1,9297300055,20150124T000000,650000.0,4.0,3.0,2950.0,5000,2,0,3,...,9,1980,970,1979,0,98126,47.5714,-122.375,2140.0,4000.0
2,1202000200,20141103T000000,233000.0,3.0,2.0,1710.0,4697,1.5,0,0,...,6,1710,0,1941,0,98002,47.3048,-122.218,1030.0,4705.0
3,8562750320,20141110T000000,580500.0,3.0,2.5,2320.0,3980,2,0,0,...,8,2320,0,2003,0,98027,47.5391,-122.070,2580.0,3980.0
4,7589200193,20141110T000000,535000.0,3.0,1.0,1090.0,3000,1.5,0,0,...,8,1090,0,1929,0,98117,47.6889,-122.375,1570.0,5080.0


In [65]:
df_house_data.shape, df_house_train.shape, df_house_test.shape

((4229, 21), (17384, 21), (4229, 21))

In [66]:
# Let's compute the mean of the House Prices in King County in 2 different ways.
np.mean(df_house_data["price"]), np.mean(df_house_train["price"]), np.mean(df_house_test["price"])

(543054.0430361788, 539366.62793373212, 543054.0430361788)

In [67]:
# #Mean Value
(17384*539366.62793373212 + 4229*543054.0430361788)/(17384+4229)

540088.1417665294

# Build a generic simple linear regression function 

In [68]:
def simple_linear_regression(input_feature, output):
    # compute the sum of input_feature and output
    sum_x = input_feature.sum()
    sum_y = output.sum()
    
    # compute the product of the output and the input_feature and its sum
    prod_xy = (output * input_feature).sum()
    
    # compute the squared value of the input_feature and its sum
    prod_xx = (input_feature * input_feature).sum()
    
    N = (output.size)
    # use the formula for the slope
    slope = ((prod_xy-((sum_x*sum_y)/N)) / (prod_xx -((sum_x*sum_x)/N)))
    
    # use the formula for the intercept
    intercept = sum_y/N - (slope*(sum_x / N))
    
    return (intercept, slope)

In [69]:
sqft_intercept, sqft_slope = simple_linear_regression(df_house_train["sqft_living"], df_house_train["price"])

In [70]:
print("Intercept: ", str(sqft_intercept))
print("Slope: ", str(sqft_slope))

Intercept:  -47116.0790729
Slope:  281.95883963


# Predicting Values

In [71]:
def get_regression_predictions(input_feature, intercept, slope):
    # calculate the predicted values:
    predicted_values = intercept + slope*input_feature
    return predicted_values

Now that we can calculate a prediction given the slope and intercept let's make a prediction. Use (or alter) the following to find out the estimated price for a house with 2650 squarefeet according to the squarefeet model we estiamted above.

**Quiz Question: Using your Slope and Intercept from (4), What is the predicted price for a house with 2650 sqft?**

In [72]:
my_house_sqft = 2650
estimated_price = get_regression_predictions(my_house_sqft, sqft_intercept, sqft_slope)
print("The estimated price for a house with %d squarefeet is $%.2f" % (my_house_sqft, estimated_price))

The estimated price for a house with 2650 squarefeet is $700074.85


# Residual Sum of Squares

Now that we have a model and can make predictions let's evaluate our model using Residual Sum of Squares (RSS). Recall that RSS is the sum of the squares of the residuals and the residuals is just a fancy word for the difference between the predicted output and the true output. 

Complete the following (or write your own) function to compute the RSS of a simple linear regression model given the input_feature, output, intercept and slope:

In [73]:
def get_residual_sum_of_squares(input_feature, output, intercept, slope):
    # First get the predictions
    predictions = get_regression_predictions(input_feature, intercept, slope)

    # then compute the residuals (since we are squaring it doesn't matter which order you subtract)
    # square the residuals and add them up
    RSS = ((output - predictions)**2).sum()

    return(RSS)

Let's test our get_residual_sum_of_squares function by applying it to the test model where the data lie exactly on a line. Since they lie exactly on a line the residual sum of squares should be zero!

In [74]:
print(get_residual_sum_of_squares(df_house_train["sqft_living"], df_house_train["price"], sqft_intercept, sqft_slope)) # should be 0.0

1.20191835418e+15


Now use your function to calculate the RSS on training data from the squarefeet model calculated above.

**Quiz Question: According to this function and the slope and intercept from the squarefeet model What is the RSS for the simple linear regression using squarefeet to predict prices on TRAINING data?**

In [75]:
rss_prices_on_sqft = get_residual_sum_of_squares(df_house_train["sqft_living"], df_house_train["price"], sqft_intercept, sqft_slope)
print("The RSS of predicting Prices based on Square Feet is : ", rss_prices_on_sqft)

The RSS of predicting Prices based on Square Feet is :  1.20191835418e+15


# Predict the squarefeet given price

What if we want to predict the squarefoot given the price? Since we have an equation y = a + b\*x we can solve the function for x. So that if we have the intercept (a) and the slope (b) and the price (y) we can solve for the estimated squarefeet (x).

Complete the following function to compute the inverse regression estimate, i.e. predict the input_feature given the output.

In [76]:
def inverse_regression_predictions(output, intercept, slope):
    # solve output = intercept + slope*input_feature for input_feature. Use this equation to compute the inverse predictions:
    estimated_feature = ((output - intercept) / slope)
    
    return estimated_feature

Now that we have a function to compute the squarefeet given the price from our simple regression model let's see how big we might expect a house that costs $800,000 to be.

**Quiz Question: According to this function and the regression slope and intercept from (3) what is the estimated square-feet for a house costing $800,000?**

In [77]:
my_house_price = 800000
estimated_squarefeet = inverse_regression_predictions(my_house_price, sqft_intercept, sqft_slope)
print("The estimated squarefeet for a house worth $%.2f is %d", (my_house_price, estimated_squarefeet))

The estimated squarefeet for a house worth $%.2f is %d (800000, 3004.3962451522766)


# New Model: estimate prices from bedrooms

We have made one model for predicting house prices using squarefeet, but there are many other features in the sales SFrame. 
Use your simple linear regression function to estimate the regression parameters from predicting Prices based on number of bedrooms. Use the training data!

In [78]:
# Estimate the slope and intercept for predicting 'price' based on 'bedrooms'

sqft_intercept, sqft_slope = simple_linear_regression(df_house_train['bedrooms'], df_house_train['price'])
print(sqft_intercept, sqft_slope)

109473.177623 127588.952934


# Test your Linear Regression Algorithm

Now we have two models for predicting the price of a house. How do we know which one is better? Calculate the RSS on the TEST data (remember this data wasn't involved in learning the model). Compute the RSS from predicting prices using bedrooms and from predicting prices using squarefeet.

**Quiz Question: Which model (square feet or bedrooms) has lowest RSS on TEST data? Think about why this might be the case.**

In [79]:
# Compute RSS when using bedrooms on TEST data:
sqft_intercept, sqft_slope = simple_linear_regression(df_house_train['bedrooms'], df_house_train['price'])
rss_prices_on_bedrooms = get_residual_sum_of_squares(df_house_test['bedrooms'], df_house_test['price'], sqft_intercept, sqft_slope)
print("The RSS of predicting Prices based on Bedrooms is : ",rss_prices_on_bedrooms)

The RSS of predicting Prices based on Bedrooms is :  4.9336458596e+14


In [80]:
# Compute RSS when using squarefeet on TEST data:
sqft_intercept, sqft_slope = simple_linear_regression(df_house_train['sqft_living'], df_house_train['price'])
rss_prices_on_sqft = get_residual_sum_of_squares(df_house_test['sqft_living'], df_house_test['price'], sqft_intercept, sqft_slope)
print("The RSS of predicting Prices based on Square Feet is : ", rss_prices_on_sqft)

The RSS of predicting Prices based on Square Feet is :  2.75402933618e+14
